In [443]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
separator = 75 * '-'

C:\Users\Nath\AppData\Local\Temp\ipykernel_8684\416754578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [444]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [445]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
print(data.shape)  # original dataset siwe
data = data.head(1000)
print(data.shape)
data.fillna("", inplace=True)
data.head()


(5964, 2)
(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [446]:
X = data["text"]
y = data["label"]

# print random email
from random import randint

def print_email(x, y, idx=None):
    try:
        print(separator)
        if idx is None:
            idx = randint(0, x.shape[0])
        print(f"[{idx}]", x[idx], "-->", y[idx])
        print(separator)
    except:
        print("Can't print email contents.")

print_email(X, y)

---------------------------------------------------------------------------
[19] <DIV><BR><STRONG>I am Hon. Mr. Hope Sithole one of the Personal Assistants to the young President Joseph Kabila of Democratic Republic of Congo (DRC), since he assumed power.Sincerely speaking, my agitation is real, and my word is my bond in this proposal. This money in question is part of the budget made by our President Mr. Joseph Kabila, in the fear that members of the United Nations (UN) could perhaps be mandated to offer their respective supports in forcing Iraq to disarm themselves. Eventually, his dreams do not match up with realities and as a result he (the President) started using the money on personal ground because nobody could question/probe his actions, as he believed.</STRONG></DIV><DIV><STRONG>As his closest personal assistant and with the help of two other officials one of whom was the then deputy Director, Safety and Security Department, we were able to escape with two boxes containing the

### Let's divide the training and test set into two partitions

In [447]:
# Your code
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# reset indexes
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# print shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(800,) (800,)
(200,) (200,)


## Data Preprocessing

In [448]:
import string
from nltk.corpus import stopwords
print("Punctuation:", string.punctuation)
print("Stopwords:", stopwords.words("english"), len(stopwords.words("english")))

# from nltk.stem.snowball import SnowballStemmer
# snowball = SnowballStemmer('english')

Punctuation: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', '

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags
- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining 
- Remove all the special characters
- Remove numbers
- Remove all single characters
- Remove single characters from the start
- Substitute multiple spaces with single space
- Remove prefixed 'b'
- Convert to Lowercase


In [449]:
# Your code

from bs4 import BeautifulSoup, Comment
import re

def clean_html_code(email_content):
    
    try:
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(email_content, 'lxml')
        
        # Remove <script> and <style> tags (inline JavaScript/CSS)
        for script_or_style in soup(['script', 'style']):
            script_or_style.extract()
        
        # Remove HTML comments
        for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
            comment.extract()
        
        # Get text content after removing all regular HTML tags
        email_content_clean = soup.get_text(separator=' ')

        # Use regex to remove any leftover tags or malformed tags
        email_content_clean = re.sub(r'<[^<>]*?>', '', email_content_clean)

        # Remove prefixed 'b' using regex BEFORE punctuation removal
        email_content_clean = re.sub(r"\bb['’]", '', email_content_clean)

        # Remove special characters
        email_content_clean = email_content_clean.translate(str.maketrans('', '', string.punctuation))

        # Remove all numbers
        email_content_clean = re.sub(r'\d+', '', email_content_clean)

        # Remove all single characters (isolated letters, symbols, etc.)
        email_content_clean = re.sub(r'\b\w\b', '', email_content_clean)  # \b is for word boundary, \w is for any word character
        
        # Remove single characters from the start of the string (if any)
        email_content_clean = re.sub(r'^\w\s', '', email_content_clean)  # Looks for a single character at the start followed by a space
        
        # Clean extra spaces caused by the above regex
        email_content_clean = re.sub(r'\s+', ' ', email_content_clean).strip()  # Replace multiple spaces with a single space

        # Convert the text to lowercase
        email_content_clean = email_content_clean.lower()

        return email_content_clean
    
    except Exception as e:
        print(e)
        return email_content


X_clean = X_train.apply(clean_html_code)
X_test_clean = X_test.apply(clean_html_code)

C:\Users\Nath\AppData\Local\Temp\ipykernel_8684\4076468216.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
C:\Users\Nath\AppData\Local\Temp\ipykernel_8684\4076468216.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(email_content, 'lxml')
C:\Users\Nath\AppData\Local\Temp\ipykernel_8684\4076468216.py:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(email_content, 'lxml')


In [450]:
# check result
idx = randint(0, len(X_train))
print_email(X_train, y_train, idx)
print_email(X_clean, y_train, idx)

---------------------------------------------------------------------------
[109] 8:15 am DEPART Private Residence 'En route State Department8:25 am ARRIVE State Department8:25 am PRESIDENTIAL DAILY BRIEFING8:30 am Secretary's Office8:30 am DAILY SMALL STAFF MEETING8:45 am Secretary's Office8:45 am DAILY SENIOR STAFF MEETING9:15 am Secretary's Conference Room9:30 am MEETING w/ERIC GOOSBY10:00 am Secretary's Office10:15 am PHOTOS10:30 am Secretary's Office *Jared Cohen Paul Narain (wife:Julianna Bentes)10:30 am OFFICE TIME11:30 pm Secretary's OfficeYale Alumni (Jake11:30 am PHONE CALL w/ KENYAN PM RAILA ODINGA11:40 am Secretary's Office12:00 pm PHONE CALL w/COLOMBIAN FM MARIA ANGELA HOLGUIN12:10 pm Secretary's Office12:15 pm UNGA MEETING12:45pm Secretary's Outer Office12:45 pm OFFICE TIME1:20pm Secretary's Office1:20 pm (t) DEPART State Department *En route Washington National Airport1:40 pm (t) ARRIVE Washington National Airport2:00 pm (t) DEPART Washington National Airport via US Airw

## Now let's work on removing stopwords
Remove the stopwords.

In [451]:
# Your code
stopwords_en = stopwords.words("english")
stopwords_en.append('u')  # remove u, abr. from you

def remove_stopwords(email_content):

    # tokenize the text by splitting on spaces
    tokens = email_content.split()
    tokens = [word for word in tokens if word not in stopwords_en]
    return ' '.join(tokens)

X_train_nostopwords = X_clean.apply(remove_stopwords)
X_test_nostopwords = X_test_clean.apply(remove_stopwords)



In [452]:
# check result
idx = randint(0, len(X_train))
print_email(X_clean, y_train, idx)
print_email(X_train_nostopwords, y_train, idx)

---------------------------------------------------------------------------
[638] im up and would like to talk about this rch and other stuff --> 0
---------------------------------------------------------------------------
---------------------------------------------------------------------------
[638] im would like talk rch stuff --> 0
---------------------------------------------------------------------------


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [453]:
# Your code
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)

def lemmatize_email(email_content):

    # tokenize the text by splitting on spaces
    tokens = email_content.split()

    # lemmatize tokens
    lem_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

    return ' '.join(lem_tokens)

X_train_lem = X_train_nostopwords.apply(lemmatize_email)
X_test_lem = X_test_nostopwords.apply(lemmatize_email)

In [454]:
# check result
idx = randint(0, len(X_train))
print_email(X_train_nostopwords, y_train, idx)
print_email(X_train_lem, y_train, idx)

---------------------------------------------------------------------------
[776] forgot tell fixed blackberry today outdated address --> 0
---------------------------------------------------------------------------
---------------------------------------------------------------------------
[776] forgot tell fix blackberry today outdated address --> 0
---------------------------------------------------------------------------


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [455]:
from collections import Counter


for doc in X_train_lem:
    tf = Counter()
    for word in doc.split():
        tf[word] +=1
    print(tf.items())

dict_items([('regard', 1), ('mr', 1), ('nelson', 1), ('smithkindly', 1), ('reply', 1), ('private', 1), ('email', 1), ('addressnelsonsmithyahoocom', 1)])
dict_items([('able', 1), ('reach', 1), ('oscar', 1), ('suppose', 1), ('send', 1), ('pdb', 1), ('receive', 1)])
dict_items([('huma', 2), ('abedin', 1), ('bim', 1), ('check', 1), ('pat', 1), ('work', 1), ('jack', 1), ('jake', 1), ('restalso', 1), ('follow', 1), ('memo', 1), ('prep', 1), ('call', 1)])
dict_items([('announce', 1), ('monday', 1), ('cant', 1), ('today', 1)])
dict_items([('bank', 6), ('africaagence', 1), ('san', 2), ('pedro', 2), ('bp', 1), ('cote', 2), ('divoirewest', 1), ('africa', 2), ('dear', 1), ('sir', 1), ('mr', 1), ('dorise', 2), ('marie', 2), ('francoisean', 1), ('accountant', 1), ('audit', 1), ('accounting', 1), ('section', 1), ('divoirewith', 1), ('due', 2), ('respect', 1), ('regard', 2), ('wish', 1), ('seek', 1), ('urgent', 1), ('assistance', 1), ('transfer', 1), ('sum', 1), ('u', 1), ('million', 2), ('dollarsseve

In [456]:
def bag_of_words(data):
    """Return the occurences of all words in a dataset"""

    tf = Counter()
    for email in data:    
        for word in email.split():
            # filter new stopwords like 'u' that appeared during cleanup / lemmatization
            if word not in stopwords_en:  
                tf[word] += 1
    
    # order words by descending occurence
    return sorted(tf.items(), key=lambda x: x[1], reverse=True)

bow_train = bag_of_words(X_train_lem)
bow_test = bag_of_words(X_test_lem)

# print top 10 words
print("Top 10 words (train):", bow_train[:10])
print(separator)
print("Top 10 words (test):", bow_test[:10])


Top 10 words (train): [('money', 804), ('account', 672), ('bank', 617), ('fund', 576), ('transfer', 449), ('business', 417), ('country', 390), ('contact', 381), ('transaction', 354), ('make', 340)]
---------------------------------------------------------------------------
Top 10 words (test): [('bank', 135), ('fund', 135), ('account', 133), ('money', 122), ('transfer', 87), ('business', 81), ('country', 75), ('million', 74), ('foreign', 71), ('transaction', 70)]


## Extra features

In [457]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_symbol_list = ["euro", "dollar", "pound", "€", "$"]
suspicious_words = ["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"]

def contains_money_symbols(text):
    return 1 if any(keyword in text for keyword in money_symbol_list) else 0

def contains_suspicious_words(text):
    return 1 if any(keyword in text for keyword in suspicious_words) else 0

X_train_df = pd.DataFrame(X_train_lem, columns=["text"])
X_test_df = pd.DataFrame(X_test_lem, columns=["text"])

X_train_df['money_mark'] = X_train_df['text'].apply(contains_money_symbols)
X_train_df['suspicious_words'] = X_train_df['text'].apply(contains_suspicious_words)
X_train_df['text_len'] = X_train_df['text'].apply(lambda x: len(x))

X_test_df['money_mark'] = X_test_df['text'].apply(contains_money_symbols)
X_test_df['suspicious_words'] = X_test_df['text'].apply(contains_suspicious_words)
X_test_df['text_len'] = X_test_df['text'].apply(lambda x: len(x))

X_train_df.head(10)

,text,money_mark,suspicious_words,text_len
0,regard mr nelson smithkindly reply private ema...,0,0,75
1,able reach oscar suppose send pdb receive,0,0,41
2,huma abedin bim check pat work jack jake resta...,0,0,76
3,announce monday cant today,0,0,26
4,bank africaagence san pedro bp san pedro cote ...,1,1,1040
5,dear friendmy proposal surprising personal con...,1,1,1929
6,soneri bank limitedsilver jubilee centercbritt...,1,1,1851
7,easy joy powerful yahoo bookmark toolbar,0,0,40
8,amrecollince addoattn ai need urgent assistanc...,1,1,1199
9,dear nancyi much want meet weve try pin schedu...,0,0,227


## How would work the Bag of Words with Count Vectorizer concept?

In [458]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Extract ham and spam messages
ham_messages = X_train_lem[y_train == 0]
spam_messages = X_train_lem[y_train == 1]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the ham and spam messages separately
ham_bow = vectorizer.fit_transform(ham_messages)
ham_feature_names = vectorizer.get_feature_names_out()

spam_bow = vectorizer.fit_transform(spam_messages)
spam_feature_names = vectorizer.get_feature_names_out()

# Sum up the word counts for ham and spam messages
ham_word_counts = ham_bow.sum(axis=0)
spam_word_counts = spam_bow.sum(axis=0)

# Convert the word counts into a list of tuples (word, count)
ham_words_freq = [(word, ham_word_counts[0, idx]) for idx, word in enumerate(ham_feature_names)]
spam_words_freq = [(word, spam_word_counts[0, idx]) for idx, word in enumerate(spam_feature_names)]

# Sort the words by frequency and get the top 10 words for both ham and spam
top_10_ham_words = sorted(ham_words_freq, key=lambda x: x[1], reverse=True)[:10]
top_10_spam_words = sorted(spam_words_freq, key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 words for ham and spam messages
print("Top 10 words in ham messages:", top_10_ham_words)
print(separator)
print("Top 10 words in spam messages:", top_10_spam_words)


Top 10 words in ham messages: [('say', 96), ('call', 92), ('state', 92), ('would', 92), ('pm', 89), ('work', 86), ('president', 84), ('percent', 76), ('secretary', 71), ('mr', 70)]
---------------------------------------------------------------------------
Top 10 words in spam messages: [('money', 800), ('account', 665), ('bank', 608), ('fund', 573), ('transfer', 447), ('business', 395), ('contact', 377), ('transaction', 354), ('country', 343), ('company', 320)]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [459]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# load the vectorizer
count_vectorizer = CountVectorizer(min_df = 1)
tfidf = TfidfTransformer(norm="l2")

# vectorize all dataset
term_freq_matrix = count_vectorizer.fit_transform(X_test_lem)
tfidf.fit(term_freq_matrix)
tf_idf_matrix = tfidf.transform(term_freq_matrix)

print("Vocabulary:", count_vectorizer.vocabulary_,'\n')
print()
print(tf_idf_matrix.todense())

# print idf values
df_idf = pd.DataFrame(tfidf.idf_, index=count_vectorizer.get_feature_names_out(), columns=["idf_weights"])

# sort ascending
df_idf_sorted = df_idf.sort_values(by=['idf_weights'])

df_idf_sorted.head(10)

Vocabulary: {'dear': 1264, 'sirc': 4279, 'wish': 5132, 'go': 2050, 'offer': 3309, 'consider': 1096, 'partnerei': 3492, 'mresamson': 3083, 'bamogu': 498, 'ecfrom': 1550, 'republic': 3977, 'sierraleone': 4251, 'west': 5089, 'africacnow': 141, 'seek': 4146, 'political': 3623, 'asylum': 426, 'senegal': 4168, 'unhcrei': 4901, 'get': 2036, 'contact': 1108, 'search': 4126, 'business': 748, 'partnere': 3491, 'detailsemy': 1366, 'late': 2671, 'father': 1806, 'dremustapha': 1505, 'manage': 2848, 'director': 1409, 'rainbow': 3827, 'gold': 2059, 'diamond': 1385, 'mining': 2999, 'company': 1009, 'ken': 2595, 'ema': 1593, 'sierra': 4250, 'leoneebut': 2713, 'kill': 2610, 'longing': 2785, 'civil': 924, 'war': 5042, 'property': 3762, 'totally': 4744, 'destroyedehoweverc': 1360, 'escape': 1680, 'precious': 3659, 'presently': 3679, 'deposit': 1331, 'safely': 4083, 'security': 4142, 'finance': 1855, 'dakare': 1227, 'saddle': 4078, 'problem': 3720, 'secure': 4141, 'trust': 4828, 'worthy': 5166, 'foreign': 

,idf_weights
fund,2.391282
money,2.411485
account,2.432104
million,2.496642
business,2.519115
contact,2.614425
state,2.614425
come,2.639743
dollar,2.639743
work,2.639743


In [503]:
# vectorize dataset and print shape
import numpy as np
tfidf_vectorizer = TfidfVectorizer(min_df = 1)

# fit vectorizer on train data only, then apply it to test data
X_train_vect = np.around(tfidf_vectorizer.fit_transform(X_train_lem).todense(), 3)
X_test_vect = np.around(tfidf_vectorizer.transform(X_test_lem).todense(), 3)

print(X_train_vect.shape)
print(X_test_vect.shape)
X_train_vect[:100]


(800, 15893)
(200, 15893)


array([[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.064, 0.128, 0.191, ..., 0.   , 0.   , 0.   ]])

## And the Train a Classifier?

In [504]:
# Your code
# Let's train a first textblop NaiveBayes classifier using the raw text as input
from textblob.classifiers import NaiveBayesClassifier

# let's pack our train dataset into a list of tuples (email, label)
train_data = list(zip(X_train_lem, y_train))

cl = NaiveBayesClassifier(train_data)
cl.show_informative_features(10)


Most Informative Features
        contains(dollar) = True                1 : 0      =    130.1 : 1.0
         contains(money) = True                1 : 0      =    112.5 : 1.0
    contains(assistance) = True                1 : 0      =     97.1 : 1.0
       contains(hundred) = True                1 : 0      =     88.5 : 1.0
          contains(bank) = True                1 : 0      =     88.0 : 1.0
      contains(transfer) = True                1 : 0      =     84.2 : 1.0
          contains(five) = True                1 : 0      =     77.4 : 1.0
          contains(fund) = True                1 : 0      =     75.0 : 1.0
      contains(document) = True                1 : 0      =     73.5 : 1.0
       contains(million) = True                1 : 0      =     67.9 : 1.0


In [505]:
# test classifier
idx = randint(0, X_test_lem.shape[0])
print("Input:", X_test_lem[idx])
print("Prediction:", cl.classify(X_test_lem[idx]))
print("True Label:", y_test[idx])

Input: sullivan jacob sunday january amhwoodwardyoure set go talk shaun spoke today confirm deal close likely low level workhappening today robinson consult dup tomorrow
Prediction: 0
True Label: 0


In [506]:
# calculate accuracy on test set
accurate_pred = 0
total_pred = X_test_lem.shape[0]

for idx in range(total_pred):
    if cl.classify(X_test_lem[idx]) == y_test[idx]:
        accurate_pred += 1

print("Prediction Accuracy:", accurate_pred * 100 / total_pred, "%")


Prediction Accuracy: 91.5 %


In [507]:
# Now let's train a MultinomialNB on vectorized data
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Multinomial Naive Bayes model
nb_classifier = MultinomialNB()

# Train the model on vectorized data
nb_classifier.fit(X_train_vect, y_train)

# Predict labels on the test set
y_pred = nb_classifier.predict(X_test_vect)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

# Print classification report
print("Classificatoin Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.925
Classificatoin Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.94       125
           1       0.84      0.99      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.92       200
weighted avg       0.93      0.93      0.93       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [13]:
# downlad and unzip dataset
import os
import zipfile

zip_file_path = "dsub-fraudulentemails.zip"

# Extracting the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

kg_train_csv = 'kg_train.csv/kg_train.csv'
kg_test_csv = 'kg_test.csv/kg_test.csv'


In [91]:
import pandas as pd

# Load the dataset into a pandas DataFrame
train_data = pd.read_csv(kg_train_csv)
test_data = pd.read_csv(kg_test_csv)

print(train_data.shape, test_data.shape)
train_data.head(10)

(5964, 2) (5964, 1)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
5,sure -- bottom line - you need a special secur...,0
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1
7,Abedin Huma <AbedinH@state.gov>Saturday Novemb...,0
8,There is an Oct 16th George Marshall event at ...,0
9,<P>1 25% for you as the account owner <BR>2 65...,1


In [93]:
# print random email
from random import randint

separator = 100 * '-'

def print_email(text, label, idx=None):
    try:
        print(separator)
        if idx is None:
            idx = randint(0, text.shape[0])
        print(f"[{idx}]", text[idx], "-->", label[idx])
        print(separator)
    except Exception as e:
        print(f"Can't print email contents: {e}")

print_email(train_data['text'], train_data['label'])

----------------------------------------------------------------------------------------------------
[5092] ATTN:President/Ceo Business Proposal,> > DEAR SIR,> > I am the manager of bills and exchange at the> foreignremittance> department of the ECOBANK Nigeria. I am writing this> letter to ask> foryour support and co-operation to carry out this> business> opportunity in my department.> We discovered an unclaimed sum of US$ 25.5 million> (Twenty-five> million five hundredthousand United States Dollar> only) in an account> that belongs to one of our customers who died a long> with his entire> family in November 1997 in plane crash in> Nigeria.Since we got> information about his death we have been expecting his> next of kin> tocome over and claim his money because we cannot> release the money> unless somebodyapplied for it as the next of kin or> relation to the> deceased as indicated in ourbanking guidelines.> > Rightaway,neither the family member nor the> relationhas appeared to> claim 

In [94]:
# preprocess data

# Check for missing values
print(train_data.isnull().sum())
print()

# Summary statistics of the dataset (if applicable)
print(train_data.describe())
print()

# Check the distribution of the target labels (SPAM/HAM)
print(train_data['label'].value_counts())  

text     0
label    0
dtype: int64

             label
count  5964.000000
mean      0.432260
std       0.495432
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000

label
0    3386
1    2578
Name: count, dtype: int64


In [95]:
# clean emails

from bs4 import BeautifulSoup, Comment
import re
import string


def clean_emails(email_content):
    
    try:
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(email_content, 'lxml')
        
        # Remove <script> and <style> tags (inline JavaScript/CSS)
        for script_or_style in soup(['script', 'style']):
            script_or_style.extract()
        
        # Remove HTML comments
        for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
            comment.extract()
        
        # Get text content after removing all regular HTML tags
        email_content_clean = soup.get_text(separator=' ')

        # Use regex to remove any leftover tags or malformed tags
        email_content_clean = re.sub(r'<[^<>]*?>', '', email_content_clean)

        # Remove prefixed 'b' using regex BEFORE punctuation removal
        email_content_clean = re.sub(r"\bb['’]", '', email_content_clean)

        # Remove special characters
        email_content_clean = email_content_clean.translate(str.maketrans('', '', string.punctuation))

        # Remove all numbers
        email_content_clean = re.sub(r'\d+', '', email_content_clean)

        # Remove all single characters (isolated letters, symbols, etc.)
        email_content_clean = re.sub(r'\b\w\b', '', email_content_clean)  # \b is for word boundary, \w is for any word character
        
        # Remove single characters from the start of the string (if any)
        email_content_clean = re.sub(r'^\w\s', '', email_content_clean)  # Looks for a single character at the start followed by a space
        
        # Clean extra spaces caused by the above regex
        email_content_clean = re.sub(r'\s+', ' ', email_content_clean).strip()  # Replace multiple spaces with a single space

        # Convert the text to lowercase
        email_content_clean = email_content_clean.lower()

        return email_content_clean
    
    except Exception as e:
        print(e)
        return email_content


train_data['cleaned_text'] = train_data['text'].apply(clean_emails)
test_data['cleaned_text'] = test_data['text'].apply(clean_emails)

C:\Users\Nath\AppData\Local\Temp\ipykernel_7156\1533637278.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
C:\Users\Nath\AppData\Local\Temp\ipykernel_7156\1533637278.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(email_content, 'lxml')
C:\Users\Nath\AppData\Local\Temp\ipykernel_7156\1533637278.py:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(email_content, 'lxml')
C:\Users\Nath\AppData\Local\Temp\ipykernel_7156\1533637278.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for co

In [99]:
# check result
idx = randint(0, len(X))
print_email(train_data['text'], y, idx)
print_email(train_data['cleaned_text'], y, idx)

----------------------------------------------------------------------------------------------------
[1205] Enjoy the beach and all best.B11.4(B)1.4(D)on: 08/21/2020 --> 0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[1205] enjoy the beach and all bestbbdon --> 0
----------------------------------------------------------------------------------------------------


In [100]:
# remove stopwords
from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")

def remove_stopwords(email_content):

    # tokenize the text by splitting on spaces
    tokens = email_content.split()
    tokens = [word for word in tokens if word not in stopwords_en]
    return ' '.join(tokens)

train_data['cleaned_text'] = train_data['text'].apply(remove_stopwords)
test_data['cleaned_text'] = test_data['text'].apply(remove_stopwords)

In [101]:
# check result
idx = randint(0, len(X))
print_email(train_data['text'], y, idx)
print_email(train_data['cleaned_text'], y, idx)

----------------------------------------------------------------------------------------------------
[529] I'm pretty sure I gave the original to lauren as soon a I received it. Doubt I made a copy but I may have cause he'd askedfor a tour. Let me check tomorrow. --> 0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[529] I'm pretty sure I gave original lauren soon I received it. Doubt I made copy I may cause he'd askedfor tour. Let check tomorrow. --> 0
----------------------------------------------------------------------------------------------------


In [102]:
# lemmatize emails

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)

def lemmatize_email(email_content):

    # tokenize the text by splitting on spaces
    tokens = email_content.split()

    # lemmatize tokens
    lem_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

    return ' '.join(lem_tokens)

train_data['cleaned_text'] = train_data['text'].apply(lemmatize_email)
test_data['cleaned_text'] = test_data['text'].apply(lemmatize_email)


In [104]:
# check result
idx = randint(0, len(X))
print_email(train_data['text'], y, idx)
print_email(train_data['cleaned_text'], y, idx)

----------------------------------------------------------------------------------------------------
[4064] Another example of "you should take the log from your own eye before criticizing the speck in your neighbor's." --> 0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[4064] Another example of "you should take the log from your own eye before criticize the speck in your neighbor's." --> 0
----------------------------------------------------------------------------------------------------


In [106]:
# model training
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Feature extraction using Bag of Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_data['cleaned_text'])  # Features
y = train_data['label']  # Labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the MultinomialNB classifier
model = MultinomialNB()

# Train the classifier
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Calculate mean F1 score using macro-average and weighted-average
macro_f1 = f1_score(y_test, y_pred, average='macro')
weighted_f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Macro-average F1 score: {macro_f1}")
print(f"Weighted-average F1 score: {weighted_f1}")


Accuracy: 0.9538977367979883
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.93      0.96       674
           1       0.92      0.98      0.95       519

    accuracy                           0.95      1193
   macro avg       0.95      0.96      0.95      1193
weighted avg       0.96      0.95      0.95      1193

Macro-average F1 score: 0.9534422507228638
Weighted-average F1 score: 0.9540405587029002
